# Prepare the Benthic Habitat data


### Import libraries and set up environment

In [4]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [5]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [ ]:
# Make sure "Carib_mol_30km" is in the project (made in EC_study_area_prep)

# Note that the non-benthic habitat layers (made in EC_habitat_data_prep) will be needed in the final step

In [ ]:
# Add the benthic habitat data to the ArcGIS project

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.RasterToGeodatabase(
    Input_Rasters=[r"C:\Users\bexg\Downloads\habitat_d_h_bottom.tif",
                   r"C:\Users\bexg\Downloads\habitat_d_s_benthic.tif",
                   r"C:\Users\bexg\Downloads\habitat_hard_shelf.tif",
                   r"C:\Users\bexg\Downloads\habitat_hard_slope.tif",
                   r"C:\Users\bexg\Downloads\habitat_rocky_reef.tif",
                   r"C:\Users\bexg\Downloads\habitat_s_t_s_bottom.tif",
                   r"C:\Users\bexg\Downloads\habitat_soft_shelf.tif",
                   r"C:\Users\bexg\Downloads\habitat_soft_slope.tif"],
    Output_Geodatabase=output_gdb,
    Configuration_Keyword=""
)


In [3]:
# Convert the rasters to polygons and clip to the study area

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

clip_feature = "Carib_mol_30km"

# Dictionary of benthic habitats
benthic_habitats_8 = {"habitat_d_h_bottom" : "\\deep_hard_Carib30", 
                      "habitat_d_s_benthic" : "\\deep_soft_Carib30",
                      "habitat_hard_shelf" : "\\shelf_hard_Carib30",
                      "habitat_soft_shelf" : "\\shelf_soft_Carib30",
                      "habitat_hard_slope" : "\\slope_hard_Carib30",
                      "habitat_soft_slope" : "\\slope_soft_Carib30",
                      "habitat_rocky_reef" : "\\shallow_hard_Carib30",
                      "habitat_s_t_s_bottom" : "\\shallow_soft_Carib30"}

def habitat_poly_clip(habitat, output_name):        
    # convert rasters to polygons in memory. Don't simplify
    polygon = arcpy.conversion.RasterToPolygon(habitat, "in_memory/polygon", "NO_SIMPLIFY")
    
    # repair geometry
    arcpy.management.RepairGeometry(polygon, "DELETE_NULL", "OGC")
    
    # clip to the 30km Caribbean study area in memory
    clipped = arcpy.analysis.Clip(polygon, clip_feature, "in_memory/clipped")
    
    # Copy the dissolved result to the output geodatabase
    arcpy.management.CopyFeatures(clipped, output_gdb + output_name)
        
    # repair geometry
    arcpy.management.RepairGeometry(output_gdb + output_name, "DELETE_NULL", "OGC")
    
# loop through the habitat layers and process them
for habitat, output_name in benthic_habitats_8.items():
    habitat_poly_clip(habitat, output_name)

In [7]:
# Add a column indicating the habitat type for each habitat dataset

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

# Dictionary of habitats
benthic_habitats_8 = {"deep_hard_Carib30" : "deep_hard",
              "deep_soft_Carib30" : "deep_soft",
              "shelf_hard_Carib30" : "shelf_hard",
              "shelf_soft_Carib30" : "shelf_soft",
              "slope_hard_Carib30" : "slope_hard",
              "slope_soft_Carib30" : "slope_soft",
              "shallow_hard_Carib30" : "shallow_hard",
              "shallow_soft_Carib30" : "shallow_soft"}

def habitat_field(habitat, habitat_name):
    #add field to indicate habitat type
    arcpy.management.CalculateField(
        in_table=habitat,
        field="Habitat",
        expression= f"'{habitat_name}'",
        expression_type="PYTHON3",
        code_block="",
        field_type="TEXT",
        enforce_domains="NO_ENFORCE_DOMAINS"
    )

# loop through the habitat layers and process them
for habitat, habitat_name in benthic_habitats_8.items():
    habitat_field(habitat, habitat_name)

In [14]:
# merge all the benthic habitats and other habitats into a single habitat layer

feature1 = "deep_hard_Carib30"
feature2 = "deep_soft_Carib30"
feature3 = "shelf_hard_Carib30"
feature4 = "shelf_soft_Carib30"
feature5 = "slope_hard_Carib30"
feature6 = "slope_soft_Carib30"
feature7 = "shallow_hard_Carib30"
feature8 = "shallow_soft_Carib30"
feature9 = "mangroves_mol_Carib30_dis"
feature10 = "saltmarsh_mol_Carib30_dis"
feature11 = "seamount_knoll_mol_Carib30_dis"
feature12 = "coral_mol_Carib30_dis"
feature13 = "seagrass_mol_Carib30_dis"

out_feature = "\\habitats_dis_merge"

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def habitat_merge():
    arcpy.management.Merge(
        inputs=[feature1, feature2, feature3, feature4, feature5, feature6, feature7, feature8, feature9, feature10, feature11, feature12, feature13],
        output= out_feature,
        field_mappings= None,
        add_source="NO_SOURCE_INFO"
    )

    # repair geometry
    arcpy.management.RepairGeometry(out_feature, "DELETE_NULL", "OGC")

habitat_merge()
            

In [15]:
#Can use this step if needed to dice the merged and dissolved habitat layer to reduce processing times

in_feature = "habitats_dis_merge"
out_feature = "\\habitats_dis_merge_40kDice"
VertLimit = 40000
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

# Dice the habitats_dis_merge layer to reduce processing times
arcpy.Dice_management (in_feature, output_gdb + out_feature, VertLimit)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\habitats_dis_merge_40kDice'>